# Ejercicio Covid-19

In [2]:
import os
import numpy as np
import pandas as pd

#Fechas de hoy(datos España) y ayer (datos inetrnacionales)
from datetime import datetime, timedelta
from datetime import date
date_today = date.today()
date_today_complete = date.today()
date_daybefore = date_today - timedelta(days=1)
date_today = date_today.strftime('%d/%m')
date_daybefore2 = date_daybefore.strftime('%d/%m')

date_today_large = date_today_complete.strftime('%Y-%m-%d')
date_daybefore_large = date_daybefore.strftime('%Y-%m-%d')
date_today_large

'2020-05-04'

## Traer los datos de Datadista

In [3]:
data_ccaa_confirmed_timeseries = pd.read_csv("https://raw.githubusercontent.com/datadista/datasets/master/COVID%2019/ccaa_covid19_casos.csv", sep = ',', encoding="utf-8")

data_ccaa_deaths_timeseries = pd.read_csv("https://raw.githubusercontent.com/datadista/datasets/master/COVID%2019/ccaa_covid19_fallecidos.csv", sep = ',', encoding="utf-8")

data_ccaa_recovered_timeseries = pd.read_csv("https://raw.githubusercontent.com/datadista/datasets/master/COVID%2019/ccaa_covid19_altas.csv", sep = ',', encoding="utf-8")

data_spain_datadista = pd.read_csv("https://raw.githubusercontent.com/datadista/datasets/master/COVID%2019/nacional_covid19.csv")


#Comprobar si están subidos los últimos datos, si no sale error es que hay datos.
data_ccaa_recovered_timeseries[date_today_large]

0      7021
1      2624
2       917
3      1294
4      1166
5      1610
6      5794
7      6900
8     21898
9       142
10     7506
11     2090
12     6234
13    37808
14      101
15     1277
16     2494
17    12155
18     2312
Name: 2020-05-04, dtype: int64

## RESUMEN: Resumen datos globales

In [21]:
# Función para obtener datos del mundo    
'''
def get_data_spain(url):    
    data_spain = pd.read_csv(url)
    data_spain = data_spain.groupby("Country/Region", as_index=False).sum()
    data_spain = data_spain.set_index("Country/Region").transpose()
    data_spain = data_spain['Spain']
    data_spain_actual = int(data_spain.iloc[-1])
    data_spain_before = int(data_spain.iloc[-2])
    data_spain_24h = data_spain_actual - data_spain_before
    return data_spain_24h, data_spain_actual
'''

def get_data_world(url):    
    data_world = pd.read_csv(url)
    data_world = data_world.groupby("Country/Region", as_index=False).sum()
    data_world = data_world.set_index("Country/Region").transpose()
    data_world_actual = int(data_world.sum(axis=1).iloc[-1])
    data_world_before = int(data_world.sum(axis=1).iloc[-2])
    data_world_24h = data_world_actual - data_world_before
    return data_world_24h, data_world_actual

data_confirmed_path = "COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
data_deaths_path = "COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv"
data_recovered_path = "COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv"


# Para el mundo
data_confirmed_world_24h, data_confirmed_world_actual = get_data_world(data_confirmed_path)
data_deaths_world_24h, data_deaths_world_actual = get_data_world(data_deaths_path)
data_recovered_world_24h, data_recovered_world_actual = get_data_world(data_recovered_path)

''' Cogiendo datos de John Hopkins
# Para España
data_confirmed_spain_24h, data_confirmed_spain_actual = get_data_spain(data_confirmed_path)
data_deaths_spain_24h, data_deaths_spain_actual = get_data_spain(data_deaths_path)
data_recovered_spain_24h, data_recovered_spain_actual = get_data_spain(data_recovered_path)
'''

# Para España actual(dataset de datadista)
data_confirmed_spain_actual = int(data_ccaa_confirmed_timeseries[date_today_large].sum())
data_recovered_spain_actual = int(data_ccaa_recovered_timeseries[date_today_large].sum())
data_deaths_spain_actual = int(data_ccaa_deaths_timeseries[date_today_large].sum())


# Para España nuevos casos 24h(dataset de datadista)
data_spain_datadista_trans = data_spain_datadista.set_index('fecha').transpose()
data_spain_datadista_confirmed = data_spain_datadista_trans.iloc[1]
data_spain_datadista_recovered = data_spain_datadista_trans.iloc[3]
data_spain_datadista_deaths = data_spain_datadista_trans.iloc[4]
data_confirmed_spain_24h = int(data_spain_datadista_confirmed[date_today_large]) - int(data_spain_datadista_confirmed[date_daybefore_large])
data_recovered_spain_24h = int(data_spain_datadista_recovered[date_today_large]) - int(data_spain_datadista_recovered[date_daybefore_large])
data_deaths_spain_24h= int(data_spain_datadista_deaths[date_today_large]) - int(data_spain_datadista_deaths[date_daybefore_large])

# Creamos dataframes

data_spain_resume_actual = pd.DataFrame({'Confirmados': data_confirmed_spain_actual,
                                    'Curados': data_recovered_spain_actual, 
                                    'Fallecidos': data_deaths_spain_actual}, 
                                   index=['Casos totales en España'])

data_spain_newcases = pd.DataFrame({'Confirmados': data_confirmed_spain_24h,
                                    'Curados': data_recovered_spain_24h, 
                                    'Fallecidos': data_deaths_spain_24h}, 
                                   index=['Nuevos casos en España (24h)'])


data_world_newcases = pd.DataFrame({'Confirmados': data_confirmed_world_24h,
                                    'Curados': data_recovered_world_24h, 
                                    'Fallecidos': data_deaths_world_24h}, 
                                   index=['Nuevos casos en el mundo (24h)'])

data_world_resume_actual = pd.DataFrame({'Confirmados': data_confirmed_world_actual,
                                    'Curados': data_recovered_world_actual, 
                                    'Fallecidos': data_deaths_world_actual}, 
                                   index=['Casos totales en el mundo'])

########### Junto los datos anteriores en una tabla
data_resume_final = pd.concat([data_world_resume_actual, data_world_newcases, data_spain_resume_actual, data_spain_newcases])

data_resume_final.to_csv('data_world_resume.csv', sep = ';')
print(data_resume_final)

Confirmados  Curados  Fallecidos
Casos totales en el mundo           3506729  1125236      247470
Nuevos casos en el mundo (24h)        79386    32124        3662
Casos totales en España              248301   121343       25428
Nuevos casos en España (24h)            356     2441         164


## MUNDO: Preparación general de datos por países

In [5]:
# Datos https://github.com/CSSEGISandData/COVID-19.git
data = pd.read_csv("COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv" )

#Saco un dataset con solo los países
data_countries = data.groupby("Country/Region", as_index=False).sum()

# Hago transposición de los datos para mover las columnas en filas 
data_trans = data_countries.set_index('Country/Region').transpose()
# Y me quedo sólo con los datos temporales y número de casos
data_time = data_trans[2:]

# Cogemos solo determinados países
data_time = data_time[['China', 'France', 'Germany', 'Iran','Italy', 'Korea, South', 'Norway','Spain', 'US', 'United Kingdom']]

#Hago listado de Countries
list_countries = data_time.columns
list_countries

#Convierto la columna fecha en datetime
from datetime import date
data_time.index = pd.to_datetime(data_time.index)
data_time.tail()


Country/Region,China,France,Germany,Iran,Italy,"Korea, South",Norway,Spain,US,United Kingdom
2020-04-29,83944.0,166543.0,161539.0,93657.0,203591.0,10765.0,7710.0,212917.0,1039909.0,166441.0
2020-04-30,83956.0,167299.0,163009.0,94640.0,205463.0,10774.0,7738.0,213435.0,1069424.0,172481.0
2020-05-01,83959.0,167305.0,164077.0,95646.0,207428.0,10780.0,7783.0,213435.0,1103461.0,178685.0
2020-05-02,83959.0,168518.0,164967.0,96448.0,209328.0,10793.0,7809.0,216582.0,1132539.0,183500.0
2020-05-03,83964.0,168925.0,165664.0,97424.0,210717.0,10801.0,7847.0,217466.0,1158040.0,187842.0


## MUNDO: Confirmados, curados y fallecidos

In [6]:
######### Confirmados

data_confirmed = pd.read_csv("COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv" )
#Saco un dataset con solo los países
data_confirmed = data_confirmed.groupby("Country/Region", as_index=False).sum()
# Hago transposición de los datos para mover las columnas en filas 
data_confirmed = data_confirmed.set_index('Country/Region').transpose()
# Y me quedo sólo con los datos temporales y número de casos
data_confirmed = data_confirmed[2:]
# Cogemos solo determinados países
data_confirmed = data_confirmed[['China', 'France', 'Germany', 'Italy','Spain', 'US', 'United Kingdom']]
data_confirmed = data_confirmed[-1:]

######### Muertos

data_deaths = pd.read_csv("COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv")
#Saco un dataset con solo los países
data_deaths = data_deaths.groupby("Country/Region", as_index=False).sum()
#Se ve que están todos y comprobado que suma
data_deaths[::]
# Hago transposición de los datos para mover las columnas en filas 
data_deaths = data_deaths.set_index('Country/Region').transpose()

# Y me quedo sólo con los datos temporales y número de casos
data_deaths = data_deaths[2:]
data_deaths.tail()
# Cogemos solo determinados países
data_deaths = data_deaths[['China', 'France', 'Germany', 'Italy','Spain', 'US', 'United Kingdom']]
data_deaths = data_deaths[-1:]

######### Recuperados

data_recovered = pd.read_csv("COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv" )
#Saco un dataset con solo los países
data_recovered = data_recovered.groupby("Country/Region", as_index=False).sum()
# Hago transposición de los datos para mover las columnas en filas 
data_recovered = data_recovered.set_index('Country/Region').transpose()
# Y me quedo sólo con los datos temporales y número de casos
data_recovered = data_recovered[2:]
# Cogemos solo determinados países
data_recovered = data_recovered[['China', 'France', 'Germany', 'Italy','Spain', 'US', 'United Kingdom']]
data_recovered = data_recovered[-1:]

######### Merge de datos

# Saco la fecha:
from datetime import datetime, timedelta
date_today_intern = date.today()
date_today_intern = date_today_intern.today() - timedelta(days=1) #Quito uno porque esta base de datos siempre muestra datos del día anterior
date_today_intern = date_today_intern.strftime('%d/%m')

#Juntar ahora datos
data_columns = ['Confirmados', 'Curados', 'Fallecidos']

data_together = pd.merge(data_confirmed, data_recovered, how='outer')
data_together = pd.merge(data_together, data_deaths, how='outer')
data_together.index = data_columns
data_together_final = data_together.transpose()
# data_together = data_together.reindex(columns =data_columns) 
# data_together = data_together.reindex(data_index)

# Exporto CSV
data_together_final.to_csv('data_world_actual.csv', sep = ';')
data_together_final

,Confirmados,Curados,Fallecidos
Country/Region,,,
China,83964.0,78684.0,4637.0
France,168925.0,50885.0,24900.0
Germany,165664.0,130600.0,6866.0
Italy,210717.0,81654.0,28884.0
Spain,217466.0,118902.0,25264.0
US,1158040.0,180152.0,67682.0
United Kingdom,187842.0,901.0,28520.0


## MUNDO: Confirmados, curados y fallecidos por habitantes de población

In [7]:
data_together

Country/Region,China,France,Germany,Italy,Spain,US,United Kingdom
Confirmados,83964.0,168925.0,165664.0,210717.0,217466.0,1158040.0,187842.0
Curados,78684.0,50885.0,130600.0,81654.0,118902.0,180152.0,901.0
Fallecidos,4637.0,24900.0,6866.0,28884.0,25264.0,67682.0,28520.0


In [8]:
pob_wuhan = 11895000
pob_china = 1393000000
pob_france = 67028048
pob_germany = 83019200
pob_italy = 60541000
pob_spain = 46940000
pob_US = 328200000
pob_UK = 66650000

# Por cada millón de habitantes hay estos casos...
data_together_den = data_together.copy()
data_together_den['China'] = (data_together_den['China'] / pob_china)*1000000 # Por cada caso infectado hay 142 que no lo están
data_together_den['France'] = (data_together_den['France'] / pob_france)*1000000 # Por cada caso infectado hay 455 que no lo están
data_together_den['Germany'] = (data_together_den['Germany'] / pob_germany)*1000000  # Por cada caso infectado hay 455 que no lo están
data_together_den['Italy'] = (data_together_den['Italy'] / pob_italy)*1000000
data_together_den['Spain'] = (data_together_den['Spain'] / pob_spain)*1000000
data_together_den['US'] = (data_together_den['US'] / pob_US)*1000000
data_together_den['United Kingdom'] = (data_together_den['United Kingdom'] / pob_UK)*1000000
data_together_den = data_together_den.round(0)
# Exporto CSV
data_together_den.to_csv('data_world_actual_den.csv', sep = ';')
data_together_den

Country/Region,China,France,Germany,Italy,Spain,US,United Kingdom
Confirmados,60.0,2520.0,1995.0,3481.0,4633.0,3528.0,2818.0
Curados,56.0,759.0,1573.0,1349.0,2533.0,549.0,14.0
Fallecidos,3.0,371.0,83.0,477.0,538.0,206.0,428.0


## MUNDO: CSV : covid19_data_drive_from100.csv

In [9]:
# Coger solo datos del último mes
data_time_30days = data_time[-200:]
# Si quiero quitar China: data_time_filter = data_time_filter.drop('China', axis=1)

#Exporto los datos en diferentes arhcivos con casa país
for country in list_countries:
    dataX = data_time_30days[country]
    dataX = dataX[dataX > 200]
    dataX.to_csv('data_' + country + ".csv", index=False, header= country)

# Lista dataframes
# dataframes_list = ['dataframe_France', 'dataframe_China', 'dataframe_Germany', 'dataframe_Iran', 'dataframe_Italy', 'dataframe_Korea', 'dataframe_Norway', 'dataframe_Spain', 'dataframe_US', 'dataframe_UnitedKingdom']

# Creamos los CSV
dataframe_France = pd.read_csv('data_France.csv')
dataframe_China = pd.read_csv('data_China.csv')
dataframe_Germany = pd.read_csv('data_Germany.csv')
dataframe_Iran = pd.read_csv('data_Iran.csv')
dataframe_Italy = pd.read_csv('data_Italy.csv')
dataframe_Korea = pd.read_csv('data_Korea, South.csv')
dataframe_Norway = pd.read_csv('data_Norway.csv')
dataframe_Spain = pd.read_csv('data_Spain.csv')
dataframe_US = pd.read_csv('data_US.csv')
dataframe_UnitedKingdom = pd.read_csv('data_United Kingdom.csv')

#Les añadimos NaN al final y reindexamos
new_index = np.arange(1,71)
dataframe_France = dataframe_France.reindex(new_index)
dataframe_China = dataframe_China.reindex(new_index)
dataframe_Germany = dataframe_Germany.reindex(new_index)
dataframe_Iran = dataframe_Iran.reindex(new_index)
dataframe_Italy = dataframe_Italy.reindex(new_index)
dataframe_Korea = dataframe_Korea.reindex(new_index)
dataframe_Norway = dataframe_Norway.reindex(new_index)
dataframe_Spain = dataframe_Spain.reindex(new_index)
dataframe_US = dataframe_US.reindex(new_index)
dataframe_UnitedKingdom = dataframe_UnitedKingdom.reindex(new_index)

#Juntar dataframes y Exportar CSV completo
data_drive_from100cases = dataframe_France.join([dataframe_China, dataframe_Germany, 
                                                 dataframe_Iran, dataframe_Italy, dataframe_Korea, 
                                                 dataframe_Norway, dataframe_Spain, dataframe_US,
                                                 dataframe_UnitedKingdom])

data_drive_from100cases.to_csv('data_world_from100.csv', sep = ';')
data_drive_from100cases.tail()

,France,China,Germany,Iran,Italy,"Korea, South",Norway,Spain,US,United Kingdom
66,NaN,81999.0,NaN,97424.0,205463.0,10752.0,NaN,NaN,NaN,NaN
67,NaN,82122.0,NaN,NaN,207428.0,10761.0,NaN,NaN,NaN,NaN
68,NaN,82198.0,NaN,NaN,209328.0,10765.0,NaN,NaN,NaN,NaN
69,NaN,82279.0,NaN,NaN,210717.0,10774.0,NaN,NaN,NaN,NaN
70,NaN,82361.0,NaN,NaN,NaN,10780.0,NaN,NaN,NaN,NaN


## MUNDO: CSV covid19_data_drive_days.csv

In [10]:
# Coger solo datos de últimos 60 días
data_time_Xdays = data_time[-30:]

#Exportar CSV por días progresivos
data_drive_lastX = data_time_Xdays.loc[:,list_countries]
data_drive_lastX.to_csv('data_world_timeline.csv', sep = ';', encoding="utf-8")
data_drive_lastX.tail()


Country/Region,China,France,Germany,Iran,Italy,"Korea, South",Norway,Spain,US,United Kingdom
2020-04-29,83944.0,166543.0,161539.0,93657.0,203591.0,10765.0,7710.0,212917.0,1039909.0,166441.0
2020-04-30,83956.0,167299.0,163009.0,94640.0,205463.0,10774.0,7738.0,213435.0,1069424.0,172481.0
2020-05-01,83959.0,167305.0,164077.0,95646.0,207428.0,10780.0,7783.0,213435.0,1103461.0,178685.0
2020-05-02,83959.0,168518.0,164967.0,96448.0,209328.0,10793.0,7809.0,216582.0,1132539.0,183500.0
2020-05-03,83964.0,168925.0,165664.0,97424.0,210717.0,10801.0,7847.0,217466.0,1158040.0,187842.0


## ESPAÑA: Resumen datos España: data_spain_newcases24h.csv

In [11]:
data_spain_datadista

,fecha,casos_total,casos_pcr,casos_test_ac,altas,fallecimientos,ingresos_uci,hospitalizados
0,2020-02-21,2,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-22,2,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-23,2,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24,2,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-25,3,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
69,2020-04-30,240121,214213.0,25908.0,112050.0,24543.0,10775.0,116660.0
70,2020-05-01,243356,215581.0,27775.0,114678.0,24824.0,10859.0,117392.0
71,2020-05-02,245761,216764.0,28997.0,117248.0,25100.0,10908.0,117948.0
72,2020-05-03,247301,217655.0,29646.0,118902.0,25264.0,10973.0,118495.0


In [14]:
# Con datos de datadista, SOLO PARA DATOS DE PAÍS
data_path_confirmed = 'casos_total'
data_path_recovered = 'altas'
data_path_deaths = 'fallecimientos'

# Evolución últimos 40 días de nuevos casos

def day_consecutive(day):
    date_cons = date.today()
    date_cons = date_cons - timedelta(days=day)
    date_cons = date_cons.strftime('%Y-%m-%d')
    return date_cons

def data_day(data_path, number_to_rest):
    data = data_spain_datadista.set_index('fecha')
    data = data[data_path]
    data = int(data.iloc[number_to_rest]) - int(data.iloc[number_to_rest-1])
    return data

data_day(data_path_confirmed, -3)

def create_dataframe(data_day_name, data_path, index_title):
    dataframe = pd.DataFrame({day_consecutive(40): data_day_name(data_path, -40),
                              day_consecutive(39): data_day_name(data_path, -39),
                              day_consecutive(38): data_day_name(data_path, -38),
                              day_consecutive(37): data_day_name(data_path, -37),
                              day_consecutive(36): data_day_name(data_path, -36),
                              day_consecutive(35): data_day_name(data_path, -35),
                              day_consecutive(34): data_day_name(data_path, -34),
                              day_consecutive(33): data_day_name(data_path, -33),
                              day_consecutive(32): data_day_name(data_path, -32),
                              day_consecutive(31): data_day_name(data_path, -31),
                              day_consecutive(30): data_day_name(data_path, -30),
                              day_consecutive(29): data_day_name(data_path, -29),
                              day_consecutive(28): data_day_name(data_path, -28),
                              day_consecutive(27): data_day_name(data_path, -27),
                              day_consecutive(26): data_day_name(data_path, -26),
                              day_consecutive(25): data_day_name(data_path, -25),
                              day_consecutive(24): data_day_name(data_path, -24),
                              day_consecutive(23): data_day_name(data_path, -23),
                              day_consecutive(22): data_day_name(data_path, -22),
                              day_consecutive(21): data_day_name(data_path, -21),
                              day_consecutive(20): data_day_name(data_path, -20),
                              day_consecutive(19): data_day_name(data_path, -19), 
                              day_consecutive(18): data_day_name(data_path, -18),
                              day_consecutive(17): data_day_name(data_path, -17),
                              day_consecutive(16): data_day_name(data_path, -16),
                              day_consecutive(15): data_day_name(data_path, -15),
                              day_consecutive(14): data_day_name(data_path, -14),
                              day_consecutive(13): data_day_name(data_path, -13),
                              day_consecutive(12): data_day_name(data_path, -12),
                              day_consecutive(11): data_day_name(data_path, -11),
                              day_consecutive(10): data_day_name(data_path, -10),
                              day_consecutive(9): data_day_name(data_path, -9),
                              day_consecutive(8): data_day_name(data_path, -8),
                              day_consecutive(7): data_day_name(data_path, -7),
                              day_consecutive(6): data_day_name(data_path, -6),
                              day_consecutive(5): data_day_name(data_path, -5),
                              day_consecutive(4): data_day_name(data_path, -4),
                              day_consecutive(3): data_day_name(data_path, -3),
                              day_consecutive(2): data_day_name(data_path, -2),
                              day_consecutive(1): data_day_name(data_path, -1),
                             }, index=index_title)
                                    
    return dataframe

data_spain_confirmed24h_datadista = create_dataframe(data_day, data_path_confirmed, ['Confirmados (24h) en España'])
data_spain_recovered24h_datadista = create_dataframe(data_day, data_path_recovered, ['Recuperados (24h) en España'])
data_spain_deaths24h_datadista = create_dataframe(data_day, data_path_deaths, ['Fallecidos (24h) en España'])
data_spain_newcases24h_datadista = pd.concat([data_spain_confirmed24h_datadista,data_spain_recovered24h_datadista,data_spain_deaths24h_datadista], axis=0)


# Exportar
data_spain_newcases24h_datadista.to_csv('data_spain_newcases24h.csv', sep = ';')
data_spain_deaths24h_datadista.to_csv('data_spain_newcases24h_deaths.csv', sep = ';')
data_spain_newcases24h_datadista

,2020-03-25,2020-03-26,2020-03-27,2020-03-28,2020-03-29,2020-03-30,2020-03-31,2020-04-01,2020-04-02,2020-04-03,...,2020-04-24,2020-04-25,2020-04-26,2020-04-27,2020-04-28,2020-04-29,2020-04-30,2020-05-01,2020-05-02,2020-05-03
Confirmados (24h) en España,8932,9151,8268,6406,5799,8146,7411,7531,7237,6657,...,3865,2788,2606,2845,3355,3135,3235,2405,1540,1000
Recuperados (24h) en España,1272,2165,2405,2323,1604,2103,3097,3486,3031,3633,...,3353,3024,2143,1673,6399,3103,2628,2570,1654,2441
Fallecidos (24h) en España,615,745,839,715,696,820,929,877,845,780,...,378,288,331,301,453,268,281,276,164,164


In [15]:
#Con datos de John Hopkins, SOLO PARA COMPARAR CON INTERNACIONALES

path_c = "COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
path_d = "COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv"
path_r = "COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv"

# Evolución últimos 40 días de nuevos casos
def day_consecutive(day):
    date_cons = date.today()
    date_cons = date_cons - timedelta(days=day)
    date_cons = date_cons.strftime('%Y-%m-%d')
    return date_cons

def data_day(data_path, number_to_rest):
    data = pd.read_csv(data_path)
    data = data.groupby("Country/Region", as_index=False).sum()
    data = data.set_index("Country/Region").transpose()
    data = data['Spain']
    data = int(data.iloc[number_to_rest]) - int(data.iloc[number_to_rest-1])
    return data

def data_day_italy(data_path, number_to_rest):
    data = pd.read_csv(data_path)
    data = data.groupby("Country/Region", as_index=False).sum()
    data = data.set_index("Country/Region").transpose()
    data = data['Italy']
    data = int(data.iloc[number_to_rest]) - int(data.iloc[number_to_rest-1])
    return data

def data_day_france(data_path, number_to_rest):
    data = pd.read_csv(data_path)
    data = data.groupby("Country/Region", as_index=False).sum()
    data = data.set_index("Country/Region").transpose()
    data = data['France']
    data = int(data.iloc[number_to_rest]) - int(data.iloc[number_to_rest-1])
    return data

def data_day_germany(data_path, number_to_rest):
    data = pd.read_csv(data_path)
    data = data.groupby("Country/Region", as_index=False).sum()
    data = data.set_index("Country/Region").transpose()
    data = data['Germany']
    data = int(data.iloc[number_to_rest]) - int(data.iloc[number_to_rest-1])
    return data

def create_dataframe(data_day_name, data_path, index_title):
    dataframe = pd.DataFrame({day_consecutive(40): data_day_name(data_path, -40),
                              day_consecutive(39): data_day_name(data_path, -39),
                              day_consecutive(38): data_day_name(data_path, -38),
                              day_consecutive(37): data_day_name(data_path, -37),
                              day_consecutive(36): data_day_name(data_path, -36),
                              day_consecutive(35): data_day_name(data_path, -35),
                              day_consecutive(34): data_day_name(data_path, -34),
                              day_consecutive(33): data_day_name(data_path, -33),
                              day_consecutive(32): data_day_name(data_path, -32),
                              day_consecutive(31): data_day_name(data_path, -31),
                              day_consecutive(30): data_day_name(data_path, -30),
                              day_consecutive(29): data_day_name(data_path, -29),
                              day_consecutive(28): data_day_name(data_path, -28),
                              day_consecutive(27): data_day_name(data_path, -27),
                              day_consecutive(26): data_day_name(data_path, -26),
                              day_consecutive(25): data_day_name(data_path, -25),
                              day_consecutive(24): data_day_name(data_path, -24),
                              day_consecutive(23): data_day_name(data_path, -23),
                              day_consecutive(22): data_day_name(data_path, -22),
                              day_consecutive(21): data_day_name(data_path, -21),
                              day_consecutive(20): data_day_name(data_path, -20),
                              day_consecutive(19): data_day_name(data_path, -19), 
                              day_consecutive(18): data_day_name(data_path, -18),
                              day_consecutive(17): data_day_name(data_path, -17),
                              day_consecutive(16): data_day_name(data_path, -16),
                              day_consecutive(15): data_day_name(data_path, -15),
                              day_consecutive(14): data_day_name(data_path, -14),
                              day_consecutive(13): data_day_name(data_path, -13),
                              day_consecutive(12): data_day_name(data_path, -12),
                              day_consecutive(11): data_day_name(data_path, -11),
                              day_consecutive(10): data_day_name(data_path, -10),
                              day_consecutive(9): data_day_name(data_path, -9),
                              day_consecutive(8): data_day_name(data_path, -8),
                              day_consecutive(7): data_day_name(data_path, -7),
                              day_consecutive(6): data_day_name(data_path, -6),
                              day_consecutive(5): data_day_name(data_path, -5),
                              day_consecutive(4): data_day_name(data_path, -4),
                              day_consecutive(3): data_day_name(data_path, -3),
                              day_consecutive(2): data_day_name(data_path, -2),
                              day_consecutive(1): data_day_name(data_path, -1),
                             }, index=index_title)
                                    
    return dataframe

'''Si se cogen los datos de España de John Hopkins
# data_spain_confirmed24h = create_dataframe(data_day, path_c, ['Confirmados (24h) en España'])
# data_spain_deaths24h = create_dataframe(data_day, path_d, ['Fallecidos (24h) en España'])
# data_spain_recovered24h = create_dataframe(data_day, path_r, ['Recuperados (24h) en España'])
#### Juntar dataframes
# data_spain_newcases24h = pd.concat([data_spain_confirmed24h,data_spain_recovered24h,data_spain_deaths24h], axis=0)
# data_spain_newcases24h.to_csv('data_spain_newcases24h.csv', sep = ';')
# data_spain_deaths24h.to_csv('data_spain_newcases24h_deaths.csv', sep = ';')
'''

######## Lo mismo en italia
data_italy_confirmed24h = create_dataframe(data_day_italy, path_c, ['Confirmados (24h) en Italia'])
data_italy_deaths24h = create_dataframe(data_day_italy, path_d, ['Fallecidos (24h) en Italia'])
data_italy_recovered24h = create_dataframe(data_day_italy, path_r, ['Recuperados (24h) en Italia'])
data_italy_newcases24h = pd.concat([data_italy_confirmed24h,data_italy_recovered24h,data_italy_deaths24h], axis=0)

######## Lo mismo en Francia
data_france_confirmed24h = create_dataframe(data_day_france, path_c, ['Confirmados (24h) en Francia'])
data_france_deaths24h = create_dataframe(data_day_france, path_d, ['Fallecidos (24h) en Francia'])
data_france_recovered24h = create_dataframe(data_day_france, path_r, ['Recuperados (24h) en Francia'])
data_france_newcases24h = pd.concat([data_france_confirmed24h,data_france_recovered24h,data_france_deaths24h], axis=0)

######## Lo mismo en Alemania
data_germany_confirmed24h = create_dataframe(data_day_germany, path_c, ['Confirmados (24h) en Alemania'])
data_germany_deaths24h = create_dataframe(data_day_germany, path_d, ['Fallecidos (24h) en Alemania'])
data_germany_recovered24h = create_dataframe(data_day_germany, path_r, ['Recuperados (24h) en Alemania'])
data_germany_newcases24h = pd.concat([data_germany_confirmed24h,data_germany_recovered24h,data_germany_deaths24h], axis=0)


###### Juntar dataframes Spain + Italy sólo de fallecidos, OJO LOS DATOS DE SPAIN SE COGEN DEL CÁLCULO ANTERIOR DE DATADISTA
data_somecountries_newcases24h_deaths = pd.concat([data_spain_deaths24h_datadista, data_italy_deaths24h,
                                                   data_france_deaths24h, data_germany_deaths24h], axis=0)
data_somecountries_newcases24h_deaths.to_csv('data_somecountries_newcases24h.csv', sep = ';')
data_somecountries_newcases24h_deaths


,2020-03-25,2020-03-26,2020-03-27,2020-03-28,2020-03-29,2020-03-30,2020-03-31,2020-04-01,2020-04-02,2020-04-03,...,2020-04-24,2020-04-25,2020-04-26,2020-04-27,2020-04-28,2020-04-29,2020-04-30,2020-05-01,2020-05-02,2020-05-03
Fallecidos (24h) en España,615,745,839,715,696,820,929,877,845,780,...,378,288,331,301,453,268,281,276,164,164
Fallecidos (24h) en Italia,683,712,919,889,756,812,837,727,760,766,...,420,415,260,333,382,323,285,269,474,174
Fallecidos (24h) en Francia,231,365,299,320,294,419,502,882,984,1122,...,390,369,242,437,367,427,289,218,166,137
Fallecidos (24h) en Alemania,49,61,75,91,100,112,130,145,187,168,...,185,117,99,150,188,153,156,113,76,54


## ESPAÑA: Evolución temporal casos confirmados

In [16]:
# Preparar Evolucion temporal confirmed
data_ccaa_confirmed_timeseries_final = data_ccaa_confirmed_timeseries.drop(columns=['cod_ine'])
data_ccaa_confirmed_timeseries_final.rename(columns = {'CCAA': 'Comunidades Autónomas'}, inplace = True)
data_ccaa_confirmed_timeseries_final = data_ccaa_confirmed_timeseries_final.set_index('Comunidades Autónomas')
data_ccaa_confirmed_timeseries_final = data_ccaa_confirmed_timeseries_final.iloc[0:19]

# Exportar Evolucion temporal confirmed
data_ccaa_confirmed_timeseries_final.to_csv('data_ccaa_evolution_confirmed.csv', sep = ';', encoding="utf-8", index=True, index_label='Comunidades Autónomas')
data_ccaa_confirmed_timeseries_final

,2020-02-21,2020-02-22,2020-02-23,2020-02-24,2020-02-25,2020-02-26,2020-02-27,2020-02-28,2020-02-29,2020-03-01,...,2020-04-25,2020-04-26,2020-04-27,2020-04-28,2020-04-29,2020-04-30,2020-05-01,2020-05-02,2020-05-03,2020-05-04
Comunidades Autónomas,,,,,,,,,,,,,,,,,,,,,
Andalucía,0,0,0,0,0,0,1,6,8,12,...,12792,12991,13128,13218,13469,13656,13914,14163,14287,14349
Aragón,0,0,0,0,0,0,0,0,0,0,...,5382,5456,5504,5557,5652,5971,6103,6164,6208,6239
Asturias,0,0,0,0,0,0,0,0,0,1,...,2546,2601,2677,2688,2797,2828,2853,2864,2884,2991
Baleares,1,1,1,1,1,1,1,1,2,2,...,1907,1917,1922,1940,1950,1955,1970,1984,1990,1991
Canarias,1,1,1,1,2,5,6,6,6,7,...,2155,2167,2178,2187,2202,2205,2206,2212,2221,2225
Cantabria,0,0,0,0,0,0,0,0,0,1,...,2315,2330,2350,2386,2439,2505,2547,2577,2589,2596
Castilla La Mancha,0,0,0,0,0,0,0,0,0,1,...,18525,18995,19286,19518,19795,20071,20516,21023,21290,21511
Castilla y León,0,0,0,0,0,0,0,0,2,2,...,18259,18684,19028,19372,19851,20312,20739,20999,21229,21379
Cataluña,0,0,0,0,0,1,2,3,5,6,...,47288,47838,48782,49185,49681,49943,50334,50877,51261,51393


## ESPAÑA: Resúmenes por comunidades autónomas

In [17]:
# SOLO HACER CUANDO LOS DATOS EN DATADISTA ESTÉN ACTUALIZADOS, SI NO, NO FUNCIONA

# Confirmados por CCAA
data_ccaa_confirmed_today = data_ccaa_confirmed_timeseries[['CCAA', date_today_large]]
data_ccaa_confirmed_today.rename(columns = {date_today_large : 'Confirmados', 'CCAA' : 'Comunidades Autónomas'}, inplace = True)
data_ccaa_confirmed_today = data_ccaa_confirmed_today.set_index('Comunidades Autónomas')
data_ccaa_confirmed_today = data_ccaa_confirmed_today.copy().iloc[0:19]

# New cases
data_ccaa_confirmed_daybefore = data_ccaa_confirmed_timeseries[['CCAA', date_daybefore_large]]
data_ccaa_confirmed_daybefore.rename(columns = {date_daybefore_large : 'Confirmados', 'CCAA' : 'Comunidades Autónomas'}, inplace = True)
data_ccaa_confirmed_daybefore = data_ccaa_confirmed_daybefore.set_index('Comunidades Autónomas')
data_ccaa_confirmed_newcases = data_ccaa_confirmed_today - data_ccaa_confirmed_daybefore
data_ccaa_confirmed_newcases = data_ccaa_confirmed_newcases.copy().iloc[0:19]
data_ccaa_confirmed_newcases.rename(columns = {'Confirmados': 'Confirmados (24h)'}, inplace = True)


# Fallecidos por CCAA
data_ccaa_deaths_today = data_ccaa_deaths_timeseries[['CCAA', date_today_large]]
data_ccaa_deaths_today.rename(columns = {date_today_large : 'Fallecidos', 'CCAA' : 'Comunidades Autónomas'}, inplace = True)
data_ccaa_deaths_today = data_ccaa_deaths_today.set_index('Comunidades Autónomas')
data_ccaa_deaths_today = data_ccaa_deaths_today.copy().iloc[0:19]

# New cases
data_ccaa_deaths_daybefore = data_ccaa_deaths_timeseries[['CCAA', date_daybefore_large]]
data_ccaa_deaths_daybefore.rename(columns = {date_daybefore_large : 'Fallecidos', 'CCAA' : 'Comunidades Autónomas'}, inplace = True)
data_ccaa_deaths_daybefore = data_ccaa_deaths_daybefore.set_index('Comunidades Autónomas')
data_ccaa_deaths_newcases = data_ccaa_deaths_today - data_ccaa_deaths_daybefore
data_ccaa_deaths_newcases = data_ccaa_deaths_newcases.copy().iloc[0:19]
data_ccaa_deaths_newcases.rename(columns = {'Fallecidos': 'Fallecidos (24h)'}, inplace = True)


# Recuperados por CCAA
data_ccaa_recovered_today = data_ccaa_recovered_timeseries[['CCAA', date_today_large]]
data_ccaa_recovered_today.rename(columns = {date_today_large : 'Curados', 'CCAA' : 'Comunidades Autónomas'}, inplace = True)
data_ccaa_recovered_today = data_ccaa_recovered_today.set_index('Comunidades Autónomas')
data_ccaa_recovered_today = data_ccaa_recovered_today.copy().iloc[0:19]
# New cases
data_ccaa_recovered_daybefore = data_ccaa_recovered_timeseries[['CCAA', date_daybefore_large]]
data_ccaa_recovered_daybefore.rename(columns = {date_daybefore_large : 'Curados', 'CCAA' : 'Comunidades Autónomas'}, inplace = True)
data_ccaa_recovered_daybefore = data_ccaa_recovered_daybefore.set_index('Comunidades Autónomas')
data_ccaa_recovered_newcases = data_ccaa_recovered_today - data_ccaa_recovered_daybefore
data_ccaa_recovered_newcases = data_ccaa_recovered_newcases.copy().iloc[0:19]
data_ccaa_recovered_newcases.rename(columns = {'Curados': 'Curados (24h)'}, inplace = True)


#Juntar los tres dataframes confirmados + curados + fallecidos
data_ccaa_confirmed_recovered_deaths = pd.concat([data_ccaa_confirmed_today, data_ccaa_recovered_today, data_ccaa_deaths_today,], axis=1)

# Juntar los tres dataframes nuevos casos
data_ccaa_newcases_confirmed_recovered_deaths = pd.concat([data_ccaa_confirmed_newcases, data_ccaa_recovered_newcases, data_ccaa_deaths_newcases], axis=1)

# Exportar por grupos
data_ccaa_confirmed_recovered_deaths.to_csv('data_ccaa_confirmed_recovered_deaths.csv', sep = ';', encoding="utf-8", index=True, index_label='Comunidades Autónomas')
data_ccaa_newcases_confirmed_recovered_deaths.to_csv('data_ccaa_newcases_confirmed_recovered_deaths.csv', sep = ';', encoding="utf-8", index=True, index_label='Comunidades Autónomas')

data_ccaa_newcases_confirmed_recovered_deaths

,Confirmados (24h),Curados (24h),Fallecidos (24h)
Comunidades Autónomas,,,
Andalucía,62,124,7
Aragón,31,66,1
Asturias,107,9,4
Baleares,1,10,0
Canarias,4,16,1
Cantabria,7,-6,2
Castilla La Mancha,221,22,25
Castilla y León,150,44,18
Cataluña,132,1566,35


## ESPAÑA: Análisis por rangos de edad

In [18]:
#Comprobar si están subidos los últimos datos, si sale 0 es que no hay datos
data_spain_ccaa_age = pd.read_csv("https://raw.githubusercontent.com/datadista/datasets/master/COVID%2019/nacional_covid19_rango_edad.csv", sep = ',')
data_spain_ccaa_age[data_spain_ccaa_age["fecha"] == date_today_large].count()

fecha                33
rango_edad           33
sexo                 33
casos_confirmados    33
hospitalizados       33
ingresos_uci         33
fallecidos           33
dtype: int64

In [19]:
data_spain_ccaa_age = pd.read_csv("https://raw.githubusercontent.com/datadista/datasets/master/COVID%2019/nacional_covid19_rango_edad.csv", sep = ',')
data_spain_ccaa_age = data_spain_ccaa_age[data_spain_ccaa_age["fecha"] == date_today_large]

def take_edad(sexo):
    data_spain_ccaa_age_all = data_spain_ccaa_age[data_spain_ccaa_age["sexo"] == sexo]
    #Para sacar datos total
    data_spain_ccaa_age_total = data_spain_ccaa_age_all.tail(1)
    data_spain_ccaa_age_total = data_spain_ccaa_age_total.iloc[:, [3,4,6]]
    data_spain_ccaa_age_total.columns = ['Confirmados','Hospitalizados', 'Fallecidos']
    #Para sacar datos concretos (de ambos, hombres y mujeres)
    data_spain_ccaa_age_concrete = data_spain_ccaa_age_all[:-1]
    data_spain_ccaa_age_concrete = data_spain_ccaa_age_concrete.iloc[:, [1,3,4,6]]
    data_spain_ccaa_age_concrete.columns = ['Rango de edad','Confirmados','Hospitalizados', 'Fallecidos']
    data_spain_ccaa_age_concrete.set_index('Rango de edad', inplace=True)
    return data_spain_ccaa_age_concrete, data_spain_ccaa_age_total


data_spain_ccaa_age_ambos_concrete, data_spain_ccaa_age_ambos_total = take_edad('ambos')
data_spain_ccaa_age_hombres_concrete, data_spain_ccaa_age_hombres_total = take_edad('hombres')
data_spain_ccaa_age_mujeres_concrete, data_spain_ccaa_age_mujeres_total = take_edad('mujeres')


data_spain_ccaa_age_ambos_total.index = ['Total ambos sexos']
data_spain_ccaa_age_hombres_total.index = ['Total hombres']
data_spain_ccaa_age_mujeres_total.index = ['Total mujeres']
data_spain_ccaa_age_ambos_concat = pd.concat([data_spain_ccaa_age_mujeres_total, data_spain_ccaa_age_hombres_total, 
                                             data_spain_ccaa_age_ambos_total], axis=0)

data_spain_ccaa_age_mujeres_concrete.to_csv('data_spain_ages_mujeres.csv', sep = ';', encoding="utf-8")
data_spain_ccaa_age_hombres_concrete.to_csv('data_spain_ages_hombres.csv', sep = ';', encoding="utf-8")
data_spain_ccaa_age_ambos_concat.to_csv('data_spain_ages_ambos.csv', sep = ';', encoding="utf-8")

print(data_spain_ccaa_age_ambos_concat)
print(data_spain_ccaa_age_hombres_concrete)

Confirmados  Hospitalizados  Fallecidos
Total mujeres           121832           38183        7238
Total hombres            95013           49276        9940
Total ambos sexos       216848           87463       17178
               Confirmados  Hospitalizados  Fallecidos
Rango de edad                                         
0-9                    401             140           1
10-19                  589             123           2
20-29                 3953             657          14
30-39                 7521            1852          35
40-49                13010            4859         113
50-59                16788            8029         405
60-69                16698           10137        1079
70-79                17046           12037        2908
80-89                14480            9206        3997
90 y +                4527            2236        1386


# Subir archivos a SERVIDOR desde FTP

In [20]:
# Ayuda: https://www.pythoncircle.com/post/668/uploading-a-file-to-ftp-server-using-python/
import csv
from ftplib import FTP
from io import StringIO

passw = pd.read_csv("pass.txt", sep = ',', encoding="utf-8")
host = "akakicreations.com"
username = passw.columns[0]
password = passw.columns[1]
ftp = FTP(host=host)
login_status = ftp.login(user=username, passwd=password)
print(login_status)

print(ftp.dir())

fp = open("data_world_resume.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("data_world_resume.csv"), fp, 1024)

fp = open("data_world_actual.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("data_world_actual.csv"), fp, 1024)


fp = open("data_world_actual_den.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("data_world_actual_den.csv"), fp, 1024)

fp = open("data_world_from100.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("data_world_from100.csv"), fp, 1024)

fp = open("data_world_timeline.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("data_world_timeline.csv"), fp, 1024)

fp = open("data_somecountries_newcases24h.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("data_somecountries_newcases.csv"), fp, 1024)

fp = open("data_spain_newcases24h.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("data_spain_newcases24h.csv"), fp, 1024)

fp = open("data_spain_newcases24h_deaths.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("data_spain_newcases24h_deaths.csv"), fp, 1024)


# Datos por CCAA
fp = open("data_ccaa_evolution_confirmed.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("data_ccaa_evolution_confirmed.csv"), fp, 1024)

fp = open("data_ccaa_evolution_deaths.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("data_ccaa_evolution_deaths.csv"), fp, 1024)

fp = open("data_ccaa_evolution_recovered.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("data_ccaa_evolution_recovered.csv"), fp, 1024)

fp = open("data_ccaa_confirmed_recovered_deaths.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("data_ccaa_confirmed_recovered_deaths.csv"), fp, 1024)

fp = open("data_ccaa_newcases_confirmed_recovered_deaths.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("data_ccaa_newcases_confirmed_recovered_deaths.csv"), fp, 1024)

# Datos por grupos de edad
fp = open("data_spain_ages_mujeres.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("data_spain_ages_mujeres.csv"), fp, 1024)

fp = open("data_spain_ages_hombres.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("data_spain_ages_hombres.csv"), fp, 1024)

fp = open("data_spain_ages_ambos.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("data_spain_ages_ambos.csv"), fp, 1024)

fp.close()



230-OK. Current restricted directory is /
230 408 Kbytes used (0%) - authorized: 51200 Kb
drwxr-xr-x    2 akakicre   akakicre         4096 May  3 11:29 .
drwxr-xr-x    2 akakicre   akakicre         4096 May  3 11:29 ..
-rw-r--r--    1 akakicre   akakicre           10 May  3 17:01 .ftpquota
-rw-r--r--    1 akakicre   akakicre       154291 Apr  3 13:34 Imagen_github.jpg
-rw-r--r--    1 akakicre   akakicre          520 May  3 17:01 data_ccaa_confirmed_recovered_deaths.csv
-rw-r--r--    1 akakicre   akakicre         6711 May  3 17:01 data_ccaa_evolution_confirmed.csv
-rw-r--r--    1 akakicre   akakicre         3603 May  3 17:01 data_ccaa_evolution_deaths.csv
-rw-r--r--    1 akakicre   akakicre         3064 May  3 17:01 data_ccaa_evolution_recovered.csv
-rw-r--r--    1 akakicre   akakicre          421 May  3 17:01 data_ccaa_newcases_confirmed_recovered_deaths.csv
-rw-r--r--    1 akakicre   akakicre         1188 May  3 17:01 data_somecountries_newcases.csv
-rw-r--r--    1 akakicre   akakicre